# Data Science Essentials: Data Cleaning
    Samuel Goldrup
    MATH 403
    11 October 2022
    

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Problem 1

The g\_t\_results.csv file is a set of parent-reported scores on their child's Gifted and Talented tests. 
The two tests, OLSAT and NNAT, are used by NYC to determine if children are qualified for gifted programs.
The OLSAT Verbal has 16 questions for Kindergardeners and 30 questions for first, second, and third graders.
The NNAT has 48 questions. 
Using this dataset, answer the following questions.


1) What column has the highest number of null values and what percent of its values are null? Print the answer as a tuple with (column name, percentage). Make sure the second value is a percent.

2) List the columns that should be numeric that aren't. Print the answer as a tuple.

3) How many third graders have scores outside the valid range for the OLSAT Verbal Score? Print the answer

4) How many data values are missing (NaN)? Print the number.

Each part is one point.

In [2]:
pd.set_option('display.max_rows', None) #we want to see all the rows baby!
#part 1
g_t = pd.read_csv("g_t_results.csv")
idxmax = g_t.isna().sum().idxmax()
perc = g_t.isna().sum()[idxmax] / len(g_t[idxmax]) #calculat a percentage
print("1:",idxmax,perc*100)
#part 2
g_t.dtypes
print("2:",g_t.columns[4:7]) #these are the columns that answer the question
#part 3
OLSAT_scores = np.array(g_t[g_t['Entering Grade Level'] == '3']['OLSAT Verbal Score'])
OLSAT_scores = OLSAT_scores.astype(int)
print("3:", sum(OLSAT_scores > 30) + sum(OLSAT_scores < 0)) #total number of scores out of the range
#part 4
print("4:",np.sum(g_t.isna().to_numpy()))

1: School Assigned 75.21367521367522
2: Index(['OLSAT Verbal Score', 'OLSAT Verbal Percentile',
       'NNAT Non Verbal Raw Score'],
      dtype='object')
3: 1
4: 192


### Problem 2

imdb.csv contains a small set of information about 99 movies. Clean the data set by doing the following in order: 

1) Remove duplicate rows by dropping the first **or** last. Print the shape of the dataframe after removing the rows.

2) Drop all rows that contain missing data. Print the shape of the dataframe after removing the rows.

3) Remove rows that have data outside valid data ranges and explain briefly how you determined your ranges for each column.

4) Identify and drop columns with three or fewer different values. Print a tuple with the names of the columns dropped.

5) Convert the titles to all lower case.

Print the first five rows of your dataframe.

In [3]:
imdb = pd.read_csv("imdb.csv")
imdb.drop_duplicates(keep='first',inplace=True)
print(imdb.shape)
imdb.dropna(inplace=True)
print(imdb.shape)

#JUSTIFICATIONS
duration_lb, duration_ub = 30,300 #I have never heard of a movie outside this range
facebook_likes_lb = 1000 #very unlikely a movie gets 0 likes on fb
budget_lb = 10000 #you probably need at least $10000 to make a movie that gets onto imdb
title_year_lb = 1887 #year before the first movie came out

#use masks that say when we go out of the bound
col_idx_duration_low = imdb[imdb['duration'] < duration_lb].index
col_idx_duration_high = imdb[imdb['duration'] > duration_ub].index
col_idx_budget_low = imdb[imdb['budget'] < budget_lb].index
col_idx_title_yr_low = imdb[imdb['title_year'] < title_year_lb].index

#maybe we're actually dropping rows and I named this poorly
imdb.drop(col_idx_duration_low,inplace=True)
imdb.drop(col_idx_duration_high,inplace=True)
imdb.drop(col_idx_budget_low,inplace=True)
imdb.drop(col_idx_title_yr_low,inplace=True)

print(imdb.shape)

#the columns we want to drop
drop_cols = list(imdb.nunique()[imdb.nunique() <= 3].index)
print(tuple(drop_cols))
imdb.drop(drop_cols,axis=1,inplace=True)
print(imdb.shape)
imdb['movie_title'] = imdb['movie_title'].str.lower()

imdb.head(5)
#explanation

(93, 13)
(64, 13)
(59, 13)
('color', 'language')
(59, 11)


,director_name,duration,gross,genres,movie_title,title_year,country,budget,imdb_score,actors,movie_facebook_likes
0,Martin Scorsese,240,116866727.0,Biography|Comedy|Crime|Drama,the wolf of wall street,2013,USA,100000000.0,8.2,"Leonardo DiCaprio,Matthew McConaughey,Jon Favreau",138000
1,Shane Black,195,408992272.0,Action|Adventure|Sci-Fi,iron man 3,2013,USA,200000000.0,7.2,"Robert Downey Jr.,Jon Favreau,Don Cheadle",95000
2,Quentin Tarantino,187,54116191.0,Crime|Drama|Mystery|Thriller|Western,the hateful eight,2015,USA,44000000.0,7.9,"Craig Stark,Jennifer Jason Leigh,Zoë Bell",114000
3,Kenneth Lonergan,186,46495.0,Drama,margaret,2011,usa,14000000.0,6.5,"Matt Damon,Kieran Culkin,John Gallagher Jr.",0
4,Peter Jackson,186,258355354.0,Adventure|Fantasy,the hobbit: the desolation of smaug,2013,USA,225000000.0,7.9,"Aidan Turner,Adam Brown,James Nesbitt",83000


See justification in comments on lines 8-11.

### Problem 3

basketball.csv contains data for all NBA players between 2001 and 2018.
Each row represents a player's stats for a year.

Create two new features:

    career_length (int): number of years player has been playing (start at 0).
    
    target (str): The target team if the player is leaving. If the player is retiring, the target should be 'retires'.
                  A player is retiring if their name doesn't exist the next year.
                  (Set the players in 2019 to NaN).

Remove all duplicate players in each year.
Remove all rows except those where a player changes team, that is, target is not null nor 'retires'.

Drop the player, year, and team_id columns.

Return the first 10 lines of your dataframe.

In [4]:
nba = pd.read_csv("basketball.csv") 
#drop duplicate player, keep the first instance of them
nba.drop_duplicates(subset=["player","year"],keep="first",inplace=True)

nba.sort_values(by="year",inplace=True) #order by year in ascending order

nba['career_length'] = 0.0 #to be changed

nba['target'] = np.nan #just following directions here
baller_names = nba["player"].unique()

for name in baller_names:
    baller_mask = nba["player"] == name #mask over each player
    num_years = baller_mask.sum() #what we change career_length to
    nba.loc[baller_mask, "career_length"] = np.arange(num_years,dtype=int)
    for i,ind in enumerate(nba[baller_mask].index):
        row = nba[baller_mask].iloc[i]
        if row["year"] == 2019:
            pass
        elif i == num_years - 1: #retirement
            nba.loc[ind,"target"] = "retires"
        elif row["team_id"] != nba[baller_mask].iloc[i+1]["team_id"]:
            nba.loc[ind,"target"] = nba[baller_mask].iloc[i+1]["team_id"]

nba = nba[~nba["target"].isna()] #filter out nans
nba.drop(columns=["player","year","team_id"],inplace=True) #just following directions
nba = nba[nba["target"] != 'retires']
nba.sort_index(inplace=True) #redo the indices

nba.iloc[:10]

,age,per,ws,bpm,career_length,target
453,27,8.2,1.0,-2.5,5.0,PHO
461,24,13.0,1.2,-0.9,2.0,ATL
462,24,15.9,6.2,2.9,3.0,MEM
464,33,12.7,3.7,-1.9,14.0,HOU
467,32,11.8,5.3,0.7,13.0,PHO
477,29,7.5,1.1,-2.8,9.0,MIN
482,31,14.1,1.9,-0.2,10.0,SAS
489,25,14.1,2.9,-2.4,6.0,CHO
490,29,12.6,2.8,0.1,2.0,SAC
493,28,13.0,0.0,-3.2,7.0,MIL


### Problem 4

Load housing.csv into a dataframe with index=0. Descriptions of the features are in housing_data_description.txt.  
The goal is to construct a regression model that predicts SalePrice using the other features of the dataset.  Do this as follows:

	1) Identify and handle the missing data.  Hint: Dropping every row with some missing data is not a good choice because it gives you an empty dataframe.  What can you do instead?
    FIXME
	2) Identify the variable with nonnumeric values that are misencoded as numbers.  One-hot encode it. Hint: don't forget to remove one of the encoded columns to prevent collinearity with the constant column (which you will add later).
    
    3) Add a constant column to the dataframe.

    4) Save a copy of the dataframe.

	5) Choose four categorical featrues that seem very important in predicting SalePrice. One-hot encode these features and remove all other categorical features.
		
	6) Run an OLS using all numerical data regression on your model.  

	
Print the ten features that have the highest coef in your model and the summary. Don't print the OLS

In [5]:
#1
housing = pd.read_csv("housing.csv",index_col=0)
housing.dropna(axis=1,thresh=len(housing)*0.06,inplace=True)
#2
housing = pd.get_dummies(housing,columns=['MSSubClass'],drop_first=True)
#3
housing["cons"] = 1
#4
housing_copy = housing.copy()
#5
for col in ['Neighborhood','MSZoning','OverallQual','OverallCond']:
    housing_copy[col] = housing_copy[col].fillna("None",inplace=True) #can't run a model on Nan
housing_copy = pd.get_dummies(housing_copy,columns=['Neighborhood','MSZoning','OverallQual','OverallCond'],drop_first=True)
#get dummies on those categoricals, even nans

#remove all other categorical features
cat_cols = [col for col in housing_copy.columns if housing_copy[col].dtype == 'object']
housing_copy.drop(columns=cat_cols,inplace=True)
housing_copy.fillna(0,inplace=True) #can't do OLS on null vals

#run OLS
y = housing_copy['SalePrice']
X = housing_copy.loc[:,housing_copy.columns != 'SalePrice']

results = sm.OLS(y,X).fit()

summary = results.summary()

results_as_html = summary.tables[1].as_html()
result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

#the ten largest coefficients (not absolute value)
result_df.sort_values(['coef'],ascending=False)[:10]

,coef,std err,t,P>|t|,[0.025,0.975]
MSSubClass_40,20700.0000,19100.000,1.086,0.278,-16700.000,58100.000
MSSubClass_45,19850.0000,11400.000,1.745,0.081,-2470.180,42200.000
GarageCars,17480.0000,3209.703,5.445,0.000,11200.000,23800.000
MSSubClass_30,7934.4901,6009.133,1.320,0.187,-3853.284,19700.000
Fireplaces,7906.0359,1898.388,4.165,0.000,4182.076,11600.000
BsmtFullBath,7082.1870,2827.513,2.505,0.012,1535.615,12600.000
FullBath,6711.2647,3037.933,2.209,0.027,751.924,12700.000
TotRmsAbvGrd,6271.2262,1334.474,4.699,0.000,3653.465,8888.988
MSSubClass_70,4509.8385,7802.471,0.578,0.563,-10800.000,19800.000
BsmtHalfBath,2742.6837,4405.857,0.623,0.534,-5900.036,11400.000


### Problem 5

Using the copy of the dataframe you created in Problem 4, one-hot encode all the categorical variables.
Print the shape of the dataframe and run OLS.

Print the ten features that have the highest coef in your model and the summary.
Write a couple of sentences discussing which model is better and why.

In [6]:
#reproduce what was done in problem 4 up until we save the copy
housing5 = pd.read_csv("housing.csv",index_col=0)

housing5.dropna(axis=1,thresh=len(housing)*0.06,inplace=True)

housing5 = pd.get_dummies(housing5,columns=['MSSubClass'],drop_first=True)

housing5["cons"] = 1

cat_vars = [col for col in housing5.columns if housing5[col].dtype == 'object']
for var in cat_vars: #remove the null values
    housing5[var] = housing5[var].fillna("None",inplace=True)
housing5 = pd.get_dummies(housing5,columns=cat_vars,drop_first=True)
#get dummies, even for the Nones
    
housing5.fillna(0,inplace=True) #can't do OLS on null values
    
print("shape", housing5.shape)
    
y = housing5['SalePrice']
X = housing5.loc[:,housing5.columns != 'SalePrice']

results = sm.OLS(y,X).fit()

summary = results.summary()

results_as_html = summary.tables[1].as_html()
result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

#the ten largest coefficients (not absolute value)
result_df.sort_values(['coef'],ascending=False)[:10]

shape (1460, 51)


,coef,std err,t,P>|t|,[0.025,0.975]
cons,128000.0000,1400000.000,0.091,0.927,-2620000.000,2870000.000
OverallQual,17080.0000,1214.739,14.057,0.000,14700.000,19500.000
GarageCars,15240.0000,2974.215,5.123,0.000,9402.167,21100.000
MSSubClass_30,7851.5542,5610.495,1.399,0.162,-3154.248,18900.000
BsmtFullBath,7774.4941,2609.476,2.979,0.003,2655.627,12900.000
MSSubClass_40,5868.6538,17600.000,0.334,0.739,-28600.000,40400.000
TotRmsAbvGrd,5174.0172,1232.726,4.197,0.000,2755.845,7592.189
OverallCond,5056.5526,1040.348,4.860,0.000,3015.758,7097.347
Fireplaces,4386.9340,1763.882,2.487,0.013,926.823,7847.045
FullBath,4141.1276,2806.832,1.475,0.140,-1364.881,9647.136


The model from problem 4 is better because it has 6 statistically significant regressors (P>|t|) < 0.05 and it is not trained on every single feature, meaning that we do not overfit to a local dataset, but we chose features that we think actually matter. This means that those features which actually matter do not lose predictive power to those that shouldn't. The Omitted Variables Bias that problem 4 is suffering from is likely to not be so big since we picked that regressors that are most important.